### Fine-tuning de un modelo preentrenado

*Este tutorial es una versión recortada y adaptada de:* https://huggingface.co/docs/transformers/v4.47.1/es/training 

El entrenamiento de un modelo partiendo de una inicialización aleatoria de los parámetros del mismo es una tarea muy costosa computacionalmente. Especialmente para modelos con arquitectura de tipo Transformers, donde los requerimientos computacionales pueden implicar el uso de cientos de GPUs durante semanas, e incluso más. Por ello, es más habitual en entornos modestos partir de una inicialización de los pesos procedente de un modelo preentrenado para una tarea más general que aquella que se quiere acometer. 

Este proceso, que tiene unos requerimientos computacionales mucho más modestos recibe el nombre de **fine-tuning**. En este caso, los pesos ya están inicializados para resolver una tarea más genérica, pero se expone durante varios ciclos de entrenamiento a un *dataset* más pequeño que el inicial, pero diseñado para que el modelo aprenda una tarea más específica. 

Este notebook va a demostrar cómo realizar dicho proceso utilizando la librería Transformers. Se va a realizar siguiendo dos vías diferentes.

La más sencilla, y la única que vamos a explorar en esta demostración, es utilizar la clase Trainer de la librería transformers. 

## Preparación del *dataset*

El primer paso para hacer fine-tuning de un modelo es descargar un dataset (o crear el nuestro propio) y prepararlo para el entrenamiento. El notebook anterior (04) nos enseñó varias técnicas para realizar este proceso utilizando la librería transformers.

In [3]:
# Descargar el dataset de review de yelp
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
# Imprimir un ejemplo del dataset
dataset["train"][100]
from transformers import AutoTokenizer
# Descargar el tokenizador de BERT
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
# Función para tokenizar los ejemplos, se aplica truncation y padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
# Aplicar la función a todo el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# Utilizar un subconjunto del dataset reducido si no queremos entrenar con todo el dataset
# Si lo utilizamos todo, se tarda mucho en entrenar
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Found cached dataset parquet (/home/ulc/es/dac/.cache/huggingface/datasets/parquet/yelp_review_full-66f1f8c8d1a2da02/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Entrenamiento con Pytorch Trainer

La librería Transformers proporciona una clase *Trainer* optimizada para el entrenamiento de modelos Transformer. Evita que tengamos que escribir nuestro propio bucle de entrenamiento. El API *Trainer* permite la configuración de una gama amplia de parámetros de configuración del entrenamiento y de características adicionales como *logging*, *gradiente accumulation* y *mixed precision*.

El primer paso es cargar tu modelo y especificar el número de etiquetas (*labels*) esperadas. En la documentación del dataset de [Yelp](https://huggingface.co/datasets/yelp_review_full#data-fields) vemos que hay 5 etiquetas.

Por defecto, los pesos se cargan en precisión total (*torch.float32*), sin importar si los datos estaban almacenados en una precisión más baja, por ejemplo *half precision* (*torch.float16*). Si queremos evitar esto, y que el modelo se cargue en la precisión en la que fue almacenado, tenemos que fijar el parámetro *torch_dtype="auto"*

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-c

## Hiperparámetros de entrenamiento

Para fijar los hiperparámetros ajustables del entrenamiento, crear una clase llamada *TrainingArguments* que contiene todos los hiperparámetros que puede ajustar además de *flags* para activar distintas opciones de entrenamiento. En esta demostración dejaremos los valores por defecto, pero puedes experimentar a cambiar distintos parámetros y ver qué efecto tienen en el entreanmiento. ([documentación TrainingArguments](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.TrainingArguments))


In [5]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

# Evaluación del modelo

El Trainer no evalúa automáticamente el rendimiento del modelo durante el entrenamiento. Tendrás que pasarle a Trainer una función para calcular y hacer un reporte de las métricas. La biblioteca de Datasets proporciona una función de accuracy simple que puedes cargar con la función load_metric (ver [este tutorial](https://huggingface.co/docs/datasets/metrics) para más información):


In [6]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

/mnt/netapp2/Store_uni/home/ulc/es/dac/mytx/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


Define la función compute en metric para calcular el accuracy de tus predicciones. Antes de pasar tus predicciones a compute, necesitas convertir las predicciones a logits (recuerda que todos los modelos de Transformers devuelven logits).

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Si quieres controlar tus métricas de evaluación durante el fine-tuning, especifica el parámetro eval_strategy en tus argumentos de entrenamiento para que el modelo tenga en cuenta la métrica de evaluación al final de cada época:

In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

## Trainer

Crea un objeto Trainer con tu modelo, argumentos de entrenamiento, datasets de entrenamiento y de prueba, y tu función de evaluación:

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

A continuación, aplica fine-tuning a tu modelo llamando train():

In [12]:
trainer.train()

/mnt/netapp2/Store_uni/home/ulc/es/dac/mytx/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.367411,0.384000
2,No log,1.034930,0.580000
3,No log,0.984887,0.586000


TrainOutput(global_step=375, training_loss=1.1327731119791666, metrics={'train_runtime': 356.644, 'train_samples_per_second': 8.412, 'train_steps_per_second': 1.051, 'total_flos': 789354427392000.0, 'train_loss': 1.1327731119791666, 'epoch': 3.0})

Existen dos vías más complicadas para realizar el mismo proceso, pero que a cambio proporcionan más control sobre el mismo.

- Fine-tuning con Keras: [Ejemplo](https://huggingface.co/docs/transformers/v4.47.1/es/training#fine-tuning-con-keras)
- Fine-tuning con Pytorch Nativo: [Ejemplo](https://huggingface.co/docs/transformers/v4.47.1/es/training#fine-tune-en-pytorch-nativo)

 